### Data Processing

In [1]:
pip install sodapy

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
# Load the dataset
csv_df = pd.read_csv('/content/DOHMH_New_York_City_Restaurant_Inspection_Results_20251027.csv',
                     quotechar='"', low_memory=False)
print(csv_df.shape)
print(csv_df.info())

(291742, 27)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291742 entries, 0 to 291741
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CAMIS                  291742 non-null  int64  
 1   DBA                    291738 non-null  object 
 2   BORO                   291742 non-null  object 
 3   BUILDING               291097 non-null  object 
 4   STREET                 291733 non-null  object 
 5   ZIPCODE                288728 non-null  float64
 6   PHONE                  291735 non-null  object 
 7   CUISINE DESCRIPTION    287880 non-null  object 
 8   INSPECTION DATE        291742 non-null  object 
 9   ACTION                 287880 non-null  object 
 10  VIOLATION CODE         285580 non-null  object 
 11  VIOLATION DESCRIPTION  285580 non-null  object 
 12  CRITICAL FLAG          291742 non-null  object 
 13  SCORE                  275436 non-null  float64
 14  GRADE                  

In [4]:
# Source: NYC Open Data API example from https://dev.socrata.com/foundry/data.cityofnewyork.us/43nn-pn8j

#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("43nn-pn8j", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [5]:
# Modify example to get all rows
limit = 50000
offset = 0
all_results = []

while True:
    print(f"Getting rows {offset} - {offset + limit}")
    batch = client.get("43nn-pn8j", limit=limit, offset=offset)

    if not batch:
        print("Returned all data")
        break

    all_results.extend(batch)
    offset += limit

    if len(batch) < limit:
        print(f"Final batch ({len(batch)} rows)")
        break

df = pd.DataFrame.from_records(all_results)
print(df.shape)
print(df.info())

Getting rows 0 - 50000
Getting rows 50000 - 100000
Getting rows 100000 - 150000
Getting rows 150000 - 200000
Getting rows 200000 - 250000
Getting rows 250000 - 300000
Final batch (41742 rows)
(291742, 31)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291742 entries, 0 to 291741
Data columns (total 31 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   camis                        291742 non-null  object
 1   dba                          291742 non-null  object
 2   boro                         291742 non-null  object
 3   building                     291109 non-null  object
 4   street                       291742 non-null  object
 5   zipcode                      288732 non-null  object
 6   phone                        291735 non-null  object
 7   inspection_date              291742 non-null  object
 8   critical_flag                291742 non-null  object
 9   record_date                  291742 non-n

In [6]:
# remove computed columns that are not in CSV file
computed_cols = [col for col in df.columns if col.startswith(":@computed_region")]
df.drop(columns=computed_cols, inplace=True, errors='ignore')
# verify the shapes are the same in CSV vs. API
print(df.shape)

(291742, 27)


In [7]:
# Example data
print(f"Initial shape (Each row is a Restaurant Citations): {df.shape}")
print("Columns:", df.columns.tolist())
print(df.info())
print(df.head(3))

Initial shape (Each row is a Restaurant Citations): (291742, 27)
Columns: ['camis', 'dba', 'boro', 'building', 'street', 'zipcode', 'phone', 'inspection_date', 'critical_flag', 'record_date', 'latitude', 'longitude', 'community_board', 'council_district', 'census_tract', 'bin', 'bbl', 'nta', 'location', 'cuisine_description', 'action', 'score', 'grade', 'grade_date', 'inspection_type', 'violation_code', 'violation_description']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291742 entries, 0 to 291741
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   camis                  291742 non-null  object
 1   dba                    291742 non-null  object
 2   boro                   291742 non-null  object
 3   building               291109 non-null  object
 4   street                 291742 non-null  object
 5   zipcode                288732 non-null  object
 6   phone                  291735 non-nu

In [8]:
print("Missing Values % of overall")
print((df.isnull().mean() * 100).sort_values(ascending=False))

Missing Values % of overall
grade_date               53.839008
grade                    51.172611
score                     5.591584
violation_code            2.116596
violation_description     2.116596
bin                       1.829356
nta                       1.338511
community_board           1.338511
council_district          1.335769
census_tract              1.335769
cuisine_description       1.326515
inspection_type           1.326515
action                    1.326515
location                  1.181523
zipcode                   1.031734
bbl                       0.323231
building                  0.216973
latitude                  0.166243
longitude                 0.166243
phone                     0.002399
record_date               0.000000
inspection_date           0.000000
critical_flag             0.000000
dba                       0.000000
camis                     0.000000
boro                      0.000000
street                    0.000000
dtype: float64


In [9]:
# missing data info
print("Missing values (%):")
print((df.isna().sum() / len(df) * 100).sort_values(ascending=False))

Missing values (%):
grade_date               53.839008
grade                    51.172611
score                     5.591584
violation_code            2.116596
violation_description     2.116596
bin                       1.829356
nta                       1.338511
community_board           1.338511
council_district          1.335769
census_tract              1.335769
cuisine_description       1.326515
inspection_type           1.326515
action                    1.326515
location                  1.181523
zipcode                   1.031734
bbl                       0.323231
building                  0.216973
latitude                  0.166243
longitude                 0.166243
phone                     0.002399
record_date               0.000000
inspection_date           0.000000
critical_flag             0.000000
dba                       0.000000
camis                     0.000000
boro                      0.000000
street                    0.000000
dtype: float64


In [10]:
# determin columns type based on Columns in https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j/about_data
date_cols = ["inspection_date", "grade_date", "record_date"]

numeric_cols = ["score", "latitude", "longitude"]

text_cols = ["dba", "camis", "boro", "building", "street", "phone", "zipcode", "cuisine_description",
             "action", "violation_code", "violation_description", "critical_flag", "grade", "inspection_type",
             "community_board", "council_district", "census_tract", "nta", "location"]

In [11]:
# clean up invalid rows where BORO = 0 (not in NYC)
invalid_boro_count = (df["boro"] == "0").sum()
print(f"Rows with invalid BORO: {invalid_boro_count:,}")
df = df[df["boro"] != "0"].copy()
print(df.shape) #after clean up

Rows with invalid BORO: 20
(291722, 27)


In [12]:
# standardize date columns
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")
        print(f"Null in {col} after standardization: {df[col].isna().sum()}")

Null in inspection_date after standardization: 0
Null in grade_date after standardization: 157051
Null in record_date after standardization: 0


In [13]:
# create additional date columns for analysis
df["inspection_year"] = df["inspection_date"].dt.year
df["inspection_month"] = df["inspection_date"].dt.month
df["inspection_season"] = pd.cut(df["inspection_date"].dt.month,
    bins=[0, 2, 5, 8, 11, 12],
    labels=["Winter", "Spring", "Summer", "Fall", "Winter"],
    right=True,
    ordered=False)
print(df[["inspection_date", "inspection_year", "inspection_month", "inspection_season"]].head())

  inspection_date  inspection_year  inspection_month inspection_season
0      1900-01-01             1900                 1            Winter
1      1900-01-01             1900                 1            Winter
2      2024-09-20             2024                 9              Fall
3      1900-01-01             1900                 1            Winter
4      1900-01-01             1900                 1            Winter


In [14]:
from datetime import date

df["inspection_date"] = df["inspection_date"].dt.date

bad_dates = df[(df["inspection_date"] == date(1900, 1, 1))]
print(f"Rows with inspection_date = 1900-01-01: {len(bad_dates)}")
bad_dates.head()

Rows with inspection_date = 1900-01-01: 3850


,camis,dba,boro,building,street,zipcode,phone,inspection_date,critical_flag,record_date,...,action,score,grade,grade_date,inspection_type,violation_code,violation_description,inspection_year,inspection_month,inspection_season
0,50160543,JAMROCK JERK,Manhattan,750,8 AVENUE,10036,7184006139,1900-01-01,Not Applicable,2025-10-27 06:00:16,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,1900,1,Winter
1,50162674,BAOBAO SLIDER,Manhattan,1,HERALD SQUARE,10001,9176565865,1900-01-01,Not Applicable,2025-10-27 06:00:16,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,1900,1,Winter
3,50159666,MR P PIZZERIA INC.,Manhattan,1032,AVENUE OF THE AMERICAS,10018,3472337733,1900-01-01,Not Applicable,2025-10-27 06:00:16,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,1900,1,Winter
4,50163432,VIA ROMA PIZZA BAR,Brooklyn,445,COURT STREET,11231,3474211144,1900-01-01,Not Applicable,2025-10-27 06:00:16,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,1900,1,Winter
5,50169409,STELLAR POKHARA INC.,Queens,7017,AUSTIN ST,11375,7185931830,1900-01-01,Not Applicable,2025-10-27 06:00:16,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,1900,1,Winter


In [15]:
df["record_date"] = df["record_date"].dt.date

same_dates = df[(df["inspection_date"] == date(1900, 1, 1)) &
                (df["record_date"] == date(2025, 10, 27))]
print(f"Rows with record_date = today : {len(same_dates)}")

# drop all the data today since inception date is invalid
df = df[df["inspection_date"] != date(1900, 1, 1)]
df.shape

Rows with record_date = today : 3850


(287872, 30)

In [16]:
# clean up text columns
def clean_text(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip()  # remove leading/trailing spaces
    x = re.sub(r'\s+', ' ', x)  # remove multiple spaces
    x = re.sub(r'[^\w\s&\'/.\-]', '', x)  # remove special characters
    return x

for col in text_cols:
    if col in df.columns:
        before_unique = df[col].astype(str).nunique(dropna=True)
        df[col] = df[col].apply(clean_text)
        after_unique = df[col].astype(str).nunique(dropna=True)
        print(f"{col} reduced from {before_unique} to {after_unique}")

dba reduced from 21068 to 21054
camis reduced from 26703 to 26703
boro reduced from 5 to 5
building reduced from 7574 to 7574
street reduced from 2312 to 2301
phone reduced from 24433 to 24433
zipcode reduced from 222 to 222
cuisine_description reduced from 90 to 90
action reduced from 5 to 5
violation_code reduced from 149 to 149
violation_description reduced from 226 to 206
critical_flag reduced from 3 to 3
grade reduced from 7 to 7
inspection_type reduced from 34 to 34
community_board reduced from 69 to 69
council_district reduced from 52 to 52
census_tract reduced from 1176 to 1176
nta reduced from 194 to 194
location reduced from 22551 to 22551


### EDA

In [17]:
num_restaurants = df["camis"].nunique()
print(f"Number of unique restaurants: {num_restaurants}")

Number of unique restaurants: 26703


In [18]:
restaurants_per_boro = df.groupby("boro")["camis"].nunique().sort_values(ascending=False)
print(f"Number of unique restaurants per borough: {restaurants_per_boro}")

Number of unique restaurants per borough: boro
Manhattan        10412
Brooklyn          6848
Queens            6109
Bronx             2346
Staten Island      988
Name: camis, dtype: int64


In [19]:
inspections_per_boro = df.groupby("boro")["camis"].count().sort_values(ascending=False)
print(f"Number of inspections per borough: {inspections_per_boro}")

Number of inspections per borough: boro
Manhattan        106184
Brooklyn          74119
Queens            71008
Bronx             26596
Staten Island      9965
Name: camis, dtype: int64


In [20]:
cuisine_counts = df.groupby("cuisine_description")["camis"].count()
top10_cuisines = cuisine_counts.sort_values(ascending=False).head(10)
print(f"Top 10 cuisine types by number of inspections: {top10_cuisines}")

Top 10 cuisine types by number of inspections: cuisine_description
American                    45382
Chinese                     28321
Coffee/Tea                  20342
Pizza                       17447
Latin American              13320
Mexican                     12022
Bakery Products/Desserts    11376
Caribbean                   10846
Japanese                    10267
Italian                      9584
Name: camis, dtype: int64
